## **Install Autogen Package**

In [ ]:
%%capture
!pip install pyautogen

## **Import Autogen**

In [ ]:
import autogen

## **Create a Config List**

In [ ]:
config_list = [
    {
        "model" : "gpt-4-1106-preview",
        "api_key" : "secret_key"
    }
]

## **llm configuration**

In [ ]:
llm_config = {
    "seed": 42,
    "config_list": config_list, # a list of OpenAI API configurations
    "temperature": 0 #temperature -> sampling
}

Seed is for caching and reproducibility

Temperature for sampling. 0- is low on creativity and 1 is high in creativity. if we need definitive answers giving 0 is good

## **Define Assistant**

In [ ]:
assistant = autogen.AssistantAgent(
    name = "coder",
    llm_config = llm_config)

The default assistant agent which capable of solving problems with code

## **Create User Proxy**

In [ ]:
TerminationMessage = lambda x: x.get("content", "").rstrip().endswith("TERMINATE")

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name = "user_proxy",
    human_input_mode= "NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=TerminationMessage,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
   system_message = "Reply TERMINATE if the task has been solved for your full satisfaction. Or reply CONTINUE if the task is not solved yet")

**human_input_mode** set to ALWAYS means get human inputs every time a message is received or set to "TERMINATE" to get human input only when a termination message is received or the number of auto reply reches the max_consecutive_auto_reply.

**use_docker** also can set to True or image name like "python:3" to use docker.

Plan is to not have any user input and the discussion will get terminated if the proxy user says “TERMINATE”.

## **Final Step**

The assistant receives a message from the user_proxy, which contains the task description

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="give me exact segmentation mask of this image with white color, use semantic segmentation, take the exact borders and save the result as mask.png https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCbLonevQIqBWh2Yh2C1BACaaoMhoJIKHedg&usqp=CAU")

user_proxy (to coder):

give me exact segmentation mask of this image with white color, use semantic segmentation, take the exact borders and save the result as mask.png https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCbLonevQIqBWh2Yh2C1BACaaoMhoJIKHedg&usqp=CAU

--------------------------------------------------------------------------------
coder (to user_proxy):

To perform semantic segmentation on the image and create a mask with exact borders, we can use a pre-trained deep learning model. One of the popular libraries for this task is `torchvision`, which works with PyTorch. We will use a pre-trained model such as DeepLabV3 which is trained on a dataset like COCO or Pascal VOC for semantic segmentation.

Here's the plan:
1. Download the image from the provided URL.
2. Load a pre-trained DeepLabV3 model from `torchvision`.
3. Preprocess the image to fit the model's input requirements.
4. Perform the segmentation on the image.
5. Create a mask where the segmented class(es) are

In [ ]:
user_proxy.send(
    recipient = assistant,
    message="take the human from image, make the background is Eiffel Tower in the image, but in the result i dont want to see semantic mask you can find eiffel tower from here: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCm038sxf7zQ09pZ9Jov5dsdoGJNh6YU7uDUza4WUTWimUmFOAIdHnBA1y57wSDVWa4QE&usqp=CAU, masked person will be same, save the result background.png"
)

user_proxy (to coder):

take the human from image, make the background is Eiffel Tower in the image, but in the result i dont want to see semantic mask you can find eiffel tower from here: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCm038sxf7zQ09pZ9Jov5dsdoGJNh6YU7uDUza4WUTWimUmFOAIdHnBA1y57wSDVWa4QE&usqp=CAU, masked person will be same, save the result background.png

--------------------------------------------------------------------------------
coder (to user_proxy):

To complete this task, we will follow these steps:

1. Use the previously generated segmentation mask to extract the human from the original image.
2. Download the image of the Eiffel Tower from the provided URL.
3. Combine the extracted human with the Eiffel Tower image to create a new image with the human in front of the Eiffel Tower.
4. Save the resulting image as 'background.png'.

Let's start by writing the Python code to accomplish this:

```python
# filename: combine_images.py
import torch
import tor